## 14.1 Preliminaries

This section introduces the problem and the terminology used
in the rest of the chapter.

### 14.1.1 Problem

The problem to be solved is to put the items of an input sequence in ascending
order; more precisely, in non-decreasing order, because items may be duplicated.
For example, if the input is (1, 3, 2, 4, 2), the output is (1, 2, 2, 3, 4).
Sorting in descending order can be done with the same algorithm,
but using the opposite comparison, so we won't consider it further.
I'll use 'ascending' to mean 'non-decreasing'
because the latter is a bit of a mouthful.

Imagine the items represent playing cards, with
a value (ace, two, ..., ten, jack, queen, king) and
a suit (clubs, diamonds, hearts, spades).
Depending on the game played, we may wish to sort just by suit, just by value,
by both, by colour (spades and clubs are black, hearts and diamonds are red)
or even by a bespoke order. For example, depending on the game, the highest
card in a suit may be the ace, the king, the seven or something else.

To allow the same items to be sorted in many different ways, we'll assume
the user provides a function that computes a key for any given item.
The problem then consists of putting items in ascending order of their keys.
The keys must be of a comparable type, like integers or strings:
otherwise it's impossible to sort them.

For example, to sort cards by value, from ace up to king,
one possible function is:

$$\text{key}(value, suit) = \begin{cases}
1       & \text{if } value = \text{ace} \\
11      & \text{if } value = \text{jack} \\
12      & \text{if } value = \text{queen} \\
13      & \text{if } value = \text{king} \\
value   & \text{otherwise}
\end{cases}$$

This key function ignores the suit.
If the ace is the highest instead of the lowest card, then
the function must return a value higher than 13 for an ace.
There are infinitely many possible functions, as long as
key(2, *suit*) < key(3, *suit*) < ... < key(king, *suit*) < key(ace, *suit*).

There are two versions of the sorting problem.
One creates a new sorted sequence.
In the following definition and the rest of this chapter,
*n* is the length of the input sequence.

**Function**: create ascending sequence \
**Inputs**: *unsorted*, a sequence; *key*, a function of object to object\
**Preconditions**: *key*(*a*) and *key*(*b*) are comparable for
any *a* and *b* in *unsorted*\
**Output**: *sorted*, a sequence \
**Postconditions**:

- *sorted* is a permutation of *unsorted*
- *key*(*sorted*[*i*]) ≤ *key*(*sorted*[*j*]) for every 0 ≤ *i* < *j* < *n*

The first postcondition states that the output has the same items as the input.
The second postcondition could be stated as
*key*(*sorted*[0]) ≤ *key*(*sorted*[1]) ≤ ... ≤ *key*(*sorted*[*n*−1]).

The second version modifies the input sequence.

**Operation**: put in ascending order \
**Input/Output**: *items*, a sequence\
**Inputs**: *key*, a function of object to object\
**Preconditions**: *key*(*a*) and *key*(*b*) are comparable for
any *a* and *b* in *items*\
**Postconditions**:

- post-*items* is a permutation of pre-*items*
- *key*(post-*items*[*i*]) ≤ *key*(post-*items*[*j*]) for every 0 ≤ *i* < *j* < *n*

The rest of this chapter assumes that sequences are represented as arrays,
so that algorithms can efficiently access any item in the sequence.

### 14.1.2 Problem instances

To test the sorting algorithms to be presented, I'll use playing cards,
each represented by a string of length&nbsp;2, e.g. 'AS' (ace of spades),
'7H' (seven of hearts) and 'TD' (ten of diamonds).

Many other representations of cards are possible.
Some are easier for a user to understand;
others make key functions easier to implement.
I chose this one because it makes tests quick to type and easy to understand.
It's up to the key function to transform a user-friendly representation into
a sortable key.
I define three key functions:

- A key function that returns the value of a card as an integer from 1 to 13
  allows us to sort the cards by ascending value.
- A key function that returns the suit of a card allows us to sort cards
  alphabetically by suit: clubs, diamonds, hearts, spades.
- A key function that returns the suit–value pair allows us to sort the cards
  first alphabetically by suit and within the same suit by ascending value,
  due to the [lexicographic comparison](../04_Iteration/04_1_sequences.ipynb#4.1.1.7-Comparison) of pairs.

In [1]:
# this code is also in m269_sorting.py


def suit(card: str) -> str:
    """Return the second character of the card.

    Preconditions: card has two characters;
    the first is 'A', '2' to '9', 'T', 'J', 'Q' or 'K'
    the second is 'C', 'D', 'H' or 'S'
    """
    return card[1]


VALUES = "A23456789TJQK"


def value(card: str) -> int:
    """Return the value of the card.

    Preconditions: as for function 'suit'
    Postconditions: the output is 1 to 13 respectively for
    'A', '2' to '9', 'T', 'J', 'Q', 'K'
    """
    for index in range(len(VALUES)):
        if VALUES[index] == card[0]:
            return index + 1  # return 1—13, not 0—12


def suit_value(card: str) -> tuple:
    """Return a tuple with the suit and value of the card.

    Preconditions: as for function 'suit'
    """
    return (suit(card), value(card))

In [2]:
suit_value("TD")

('D', 10)

I can now write a few tests, using Python lists for the sequences.
These tests will only work for the first version of the problem,
where the sorting function returns a sorted sequence.

In [3]:
# this code is also in m269_sorting.py

UP_DOWN = ["AS", "3H", "QD", "KC"]  # ascending values, descending suits
SAME_VALUE = ["TD", "TS", "TH", "TC"]

sorting_tests = [
    # case,        unsorted,           key, sorted
    ('empty list', [],          suit_value, []),
    ('1 card',     ['AS'],      suit_value, ['AS']),
    ('same cards', ['6D','6D'], suit_value, ['6D','6D']),
    ('3 cards',    ['JC','8D','TS'], value, ['8D','TS','JC']),
    ('values up',  UP_DOWN,          value, UP_DOWN),
    ('suits down', UP_DOWN,           suit, ['KC','QD','3H','AS']),
    ('same value', SAME_VALUE,  suit_value, ['TC','TD','TH','TS']),
]

To check the test table, I must tell `check_tests` that the 'key' argument is a function.
In Python, functions are objects of type `Callable`, defined in the `typing` module.

In [4]:
from typing import Callable
from algoesup import check_tests

check_tests(sorting_tests, [list, Callable, list])

OK: the test table passed the automatic checks.


To measure the performance of sorting algorithms we need long sequences,
but there are only 52 cards. I'll use long sequences of integers.
Integers can be compared, so a key function
that returns the item itself suffices.
A function that returns its input is called the identity function.

In [5]:
# this code is also in m269_sorting.py


def identity(item: object) -> object:
    """Return the item, i.e. the key is the whole item."""
    return item

### 14.1.3 Algorithms

An algorithm is **in-place** if it doesn't use any additional memory,
other than the call stack and a fixed number of local variables
for individual items in the sequence, for indices, Booleans, etc.
For example, finding a pair of items that add up to a given amount can be done
in-place with an exhaustive search (nested loop) for all pairs, or
not in-place, with an additional map as in an [earlier exercise](../09_Practice-1/09_3_voucher.ipynb#9.3-Voucher).

Sorting algorithms that return a new sorted sequence aren't in-place.
Those that modify the input sequence are usually in-place, but if an algorithm
would create a temporary new sorted sequence and then copy the items
to the input sequence, it wouldn't be in-place.

### 14.1.4 Sorting in Python

You've seen in [Chapter&nbsp;4](../04_Iteration/04_6_lists.ipynb#4.6-Lists) that Python has a `sorted` function which
returns a sorted list from a given sequence and
a list method `sort` which modifies the list.
The method sorts the input sequence in-place; the function doesn't.
Both can take an argument indicating which key function to use.

In [6]:
items = ["2S", "AS", "2D", "AD"]
items.sort(key=suit_value)
items

['AD', '2D', 'AS', '2S']

Like for the `reverse` parameter, omitting the parameter name leads to an error.

In [7]:
items.sort(suit_value)

TypeError: sort() takes no positional arguments

Python uses the Powersort algorithm, an improvement over Timsort, which in turn is
derived from the insertion sort and merge sort algorithms explained in later sections.
Powersort and Timsort are in-place algorithms with linear complexity
in the best case and log-linear complexity in the worst case.
You're unlikely to need another algorithm for sorting arrays in memory.
I won't explain Powersort, Timsort or how to sort data that doesn't fit in memory.
This chapter isn't about the best way of sorting; instead it presents
algorithms that illustrate previous concepts and techniques.

⟵ [Previous section](14-introduction.ipynb) | [Up](14-introduction.ipynb) | [Next section](14_2_bogosort.ipynb) ⟶